In [245]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [246]:
import re
import string
import numpy as np
import unicodedata
import pandas as pd
from tqdm import tqdm

# Register tqdm with pandas for progress_apply
tqdm.pandas()

def clean_string(cell):
    """
    Clean a single string cell by applying these rules:
      1. Strip trailing/leading whitespace.
      2. Normalize accented characters to non-accented equivalents.
      3. Remove punctuation.
      4. Remove any special characters (non-alphanumeric, non-space).
      5. If the cleaned cell consists solely of digits or a combination of the characters '_', '.', '*',
         return np.nan.
      6. If the cell contains an address (assumed to be comma-separated) followed by a city/country,
         keep only the last token.
      8. Remove any single (') or double (") quotes.
      9. Remove any backslashes (\).
    """
    if cell is None or (isinstance(cell, float) and np.isnan(cell)):
        return cell
    if not isinstance(cell, str):
        cell = str(cell)
    
    # 1. Strip whitespace.
    cell = cell.strip()
    
    # 2. Normalize accented characters to ASCII.
    cell = unicodedata.normalize('NFKD', cell).encode('ascii', 'ignore').decode('ascii')
    
    # 8 & 9. Remove quotes and backslashes.
    cell = cell.replace('"', '').replace("'", "").replace("\\", "")
    
    # 4 & 5. If the cell contains only digits or the symbols _ . *, set it to np.nan.
    if re.fullmatch(r'[\d_.*]+', cell):
        return np.nan
    
    # 3. Remove punctuation.
    translator = str.maketrans('', '', string.punctuation)
    cell = cell.translate(translator)
    
    # 4. Remove any remaining special characters (anything not alphanumeric or whitespace).
    cell = re.sub(r'[^A-Za-z0-9\s]', '', cell)
    
    # 6. If the cell appears to contain an address (comma-separated), keep only the last token.
    if ',' in cell:
        cell = cell.split(',')[-1].strip()
    
    if cell == "":
        return np.nan
    
    return cell

def fingerprint(s):
    """
    Create a canonical fingerprint for a string by:
      - Lower-casing
      - Splitting into tokens
      - Sorting the tokens alphabetically
      - Joining them back together
    This groups strings that have the same set of words in a different order.
    """
    tokens = s.lower().split()
    tokens.sort()
    return " ".join(tokens)

def standardize_series_fast(series):
    """
    Standardize a cleaned pandas Series of strings using fingerprint grouping.
    
    For each unique cleaned value, compute a fingerprint. Then group values by fingerprint and
    assign a canonical value for each group (the most frequent one in the original series).
    
    This approach is much faster than pairwise fuzzy matching for a large number of unique values.
    """
    # Build a mapping from fingerprint to list of original values
    fp_to_values = {}
    for val in series.dropna().unique():
        fp = fingerprint(val)
        fp_to_values.setdefault(fp, []).append(val)
    
    # Frequency count for each cleaned value
    freq = series.value_counts().to_dict()
    
    # For each fingerprint group, choose the canonical value (most frequent)
    canonical_map = {}
    for fp, values in fp_to_values.items():
        canonical = max(values, key=lambda x: freq.get(x, 0))
        for v in values:
            canonical_map[v] = canonical
    
    # Map each value in the series to its canonical value
    return series.map(lambda x: canonical_map.get(x, x))

def clean_and_standardize_series(series):
    """
    Clean and standardize a pandas Series of strings using a fingerprint-based grouping approach.
    
    Parameters:
      series (pd.Series): Input string column.
      
    Returns:
      pd.Series: Cleaned and standardized series.
    """
    # Clean the series with a progress bar
    cleaned = series.progress_apply(clean_string)
    # Standardize using the fast fingerprint-based method
    standardized = standardize_series_fast(cleaned)
    return standardized

<>:12: SyntaxWarning: invalid escape sequence '\)'
<>:12: SyntaxWarning: invalid escape sequence '\)'
/var/folders/47/9pg_kxkn06569ngv_cfw7zmc0000gn/T/ipykernel_91817/1764809871.py:12: SyntaxWarning: invalid escape sequence '\)'
  """


# Books.csv

In [247]:
books = pd.read_csv('BX_Books.csv', delimiter=';', encoding='latin-1').drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)

In [248]:
pd.DataFrame(books.nunique() / books.count(axis=0), columns=['Pct Unique'])

,Pct Unique
ISBN,1.000000
Book-Title,0.892309
Book-Author,0.375960
Year-Of-Publication,0.000427
Publisher,0.061929


In [249]:
pd.DataFrame(books.isna().sum(), columns=['Num Missing'])

,Num Missing
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2


In [250]:
books.dropna(inplace=True)

In [251]:
books.describe(include='all')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
count,271375,271375,271375,271375.000000,271375
unique,271375,242151,102026,NaN,16805
top,0195153448,Selected Poems,Agatha Christie,NaN,Harlequin
freq,1,27,632,NaN,7536
mean,NaN,NaN,NaN,1959.755468,NaN
std,NaN,NaN,NaN,258.013219,NaN
min,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,1989.000000,NaN
50%,NaN,NaN,NaN,1995.000000,NaN
75%,NaN,NaN,NaN,2000.000000,NaN


In [252]:
def fix_books(s):
    s = s.strip()
    
    s.replace("\\", "")

    # If "N/A" (in any case) is present, return NaN so we can impute later
    if 'N/A' in s.upper():
        return np.nan

    # Remove trailing quotes (e.g. SWEDEN" -> SWEDEN)
    s = re.sub(r'"+$', '', s)
    # Remove leading quotes (rare, but just in case)
    s = re.sub(r'^"+', '', s)
    # Replace any remaining quotes in the middle
    s = s.replace('"', '')
    # Strip once more
    s = s.strip()
    return s

books['Book-Title'] = clean_and_standardize_series(books['Book-Title'].apply(fix_books))
books['Book-Author'] = clean_and_standardize_series(books['Book-Author'].apply(fix_books))
books['Publisher'] = clean_and_standardize_series(books['Publisher'].apply(fix_books))

100%|██████████| 271375/271375 [00:00<00:00, 516737.35it/s]


# Book-Ratings.csv

In [253]:
book_ratings = pd.read_csv('BX-Book-Ratings.csv', delimiter=';', encoding='latin-1')

In [254]:
book_ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [255]:
import re
import numpy as np
import pandas as pd

def clean_isbn_string(isbn):
    """
    Clean a single ISBN string by removing characters that are not allowed.
    
    Steps:
      1. Convert to string if needed and strip leading/trailing whitespace.
      2. Remove common ISBN prefixes such as 'ISBN:' (case-insensitive).
      3. Convert to uppercase (to ensure 'x' becomes 'X').
      4. Remove any characters that are not digits, hyphen ('-'), or 'X'.
      5. Collapse multiple consecutive hyphens into a single hyphen.
      6. Remove any leading or trailing hyphens.
      
    Parameters:
      isbn (str): The ISBN string to clean.
      
    Returns:
      str or np.nan: The cleaned ISBN string, or np.nan if the result is empty.
    """
    if isbn is None or (isinstance(isbn, float) and np.isnan(isbn)):
        return np.nan
    isbn = str(isbn).strip()
    
    # Remove a common prefix (e.g., "ISBN:"), ignoring case
    isbn = re.sub(r'(?i)^ISBN[\s:]*', '', isbn)
    
    # Convert to uppercase (for example, to get 'X' instead of 'x')
    isbn = isbn.upper()
    
    # Remove any characters that are not digits, hyphen, or 'X'
    isbn = re.sub(r'[^0-9\-X]', '', isbn)
    
    # Collapse multiple hyphens into a single hyphen
    isbn = re.sub(r'-+', '-', isbn)
    
    # Remove leading/trailing hyphens
    isbn = isbn.strip('-')
    
    if isbn == "":
        return np.nan
    return isbn

def clean_isbn(series):
    """
    Clean a pandas Series of ISBN values by applying clean_isbn_string to each element.
    
    This function processes each ISBN, ensuring that only valid characters remain.
    It does not attempt to aggregate similar values, so distinct ISBNs stay distinct.
    
    Parameters:
      series (pd.Series): A pandas Series containing ISBN values.
      
    Returns:
      pd.Series: A Series with the cleaned ISBN strings.
    """
    return series.apply(clean_isbn_string)

In [256]:
book_ratings['ISBN'] = clean_isbn(book_ratings['ISBN'])

In [257]:
pd.DataFrame(book_ratings.nunique() / book_ratings.count(axis=0), columns=['Pct Unique'])

,Pct Unique
User-ID,0.091568
ISBN,0.295387
Book-Rating,0.000010


In [258]:
pd.DataFrame(book_ratings.isna().sum(), columns=['Num Missing'])

,Num Missing
User-ID,0
ISBN,55
Book-Rating,0


In [259]:
# Drop those with missing ISBN
book_ratings.dropna(inplace=True)

In [260]:
pd.DataFrame(book_ratings.isna().sum(), columns=['Num Missing'])

,Num Missing
User-ID,0
ISBN,0
Book-Rating,0


# Users.csv

In [261]:
users = pd.read_csv('BX-Users.csv', delimiter=';', encoding='latin-1')

In [262]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [263]:
pd.DataFrame(users.nunique() / users.count(axis=0), columns=['Pct Unique'])

,Pct Unique
User-ID,1.000000
Location,0.205621
Age,0.000982


In [264]:
# Clean up weird quoting in the 'Location' column
def fix_location_string(loc):
    """
    Removes trailing quotes, removes \"N/A\,
    and normalizes repeated quotes in the location field.
    """
    s = str(loc)
    # Remove the literal substring \"N/A\
    s = s.replace('\\"N/A\\', '')
    # Remove trailing double quotes (e.g. 'SRI LANKA"' -> 'SRI LANKA')
    s = re.sub(r'"+$', '', s)
    # Collapse any repeated quotes ("" or more) into a single space
    s = re.sub(r'"{2,}', ' ', s)
    # Strip leading/trailing whitespace
    return s.strip()

users['Location'] = users['Location'].apply(fix_location_string)

<>:3: SyntaxWarning: invalid escape sequence '\,'
<>:3: SyntaxWarning: invalid escape sequence '\,'
/var/folders/47/9pg_kxkn06569ngv_cfw7zmc0000gn/T/ipykernel_91817/57824958.py:3: SyntaxWarning: invalid escape sequence '\,'
  """


In [265]:
def process_location(loc):
    # Ensure we have a string
    loc = str(loc)
    
    # 1. Remove literal \N/A"" sequences (or replace them with a space)
    loc = loc.replace(r'\N/A""', ' ')
    
    # 2. Remove trailing quotes if they remain (e.g. 'SWEDEN"')
    loc = re.sub(r'"+$', '', loc)
    
    # 3. Now split on commas
    parts = [p.strip() for p in loc.lower().split(',')]
    if len(parts) >= 2:
        city, country = parts[0], parts[-1]
        
        # Standardize city
        city_mapping = {
            'nyc': 'New York',
            'new york city': 'New York',
            # add more common mappings if needed
        }
        city_std = city_mapping.get(city, city.title())
        
        # Standardize country
        country_mapping = {
            'usa': 'USA',
            'us': 'USA',
            'united states': 'USA'
            # add more common mappings if needed
        }
        country_std = country_mapping.get(country, country.upper())
        
        return pd.Series([city_std, country_std])
    else:
        # If the location doesn't have at least two parts, return NaNs
        return pd.Series([np.nan, np.nan])

# Apply the function and create new 'City' and 'Country' columns
users[['City', 'Country']] = users['Location'].apply(process_location)
users.drop(['Location'], axis=1, inplace=True)


In [266]:
def fix_city_or_country(val):
    """
    1. Convert val to string.
    2. If 'N/A' (case-insensitive) is anywhere in val, treat the whole cell as NaN.
    3. Otherwise remove any extra quotes (leading, trailing, doubled).
    4. Return cleaned string or NaN if empty after cleanup.
    """
    s = str(val).strip()
    
    # If "N/A" (in any case) is present, return NaN so we can impute later
    if 'N/A' in s.upper():
        return np.nan

    # Remove trailing quotes (e.g. SWEDEN" -> SWEDEN)
    s = re.sub(r'"+$', '', s)
    # Remove leading quotes (rare, but just in case)
    s = re.sub(r'^"+', '', s)
    # Replace any remaining quotes in the middle
    s = s.replace('"', '')
    # Strip once more
    s = s.strip()
    
    # If string is now empty, return NaN
    return s if s else np.nan

users['City'] = users['City'].apply(fix_city_or_country)
users['Country'] = users['Country'].apply(fix_city_or_country)

In [267]:
pd.DataFrame(users.nunique() / users.count(axis=0), columns=['Pct Unique'])

,Pct Unique
User-ID,1.000000
Age,0.000982
City,0.117691
Country,0.002272


In [268]:
pd.DataFrame(users.isna().sum() / users.shape[0], columns=['Pct Missing'])

,Pct Missing
User-ID,0.000000
Age,0.397199
City,0.002040
Country,0.016525


In [269]:
users['Country'] = clean_and_standardize_series(users['Country'])
users['City'] = clean_and_standardize_series(users['City'])

100%|██████████| 278858/278858 [00:00<00:00, 595006.62it/s]


### Age is missing for almost 40% of users. Imputation will be poor, but it might be better than just dropping the column.

In [270]:
import pandas as pd
import numpy as np
from sklearn.impute import IterativeImputer, SimpleImputer

# Define the columns
numeric_cols = ['User-ID', 'Age']
categorical_cols = ['City', 'Country']

# 1) Separate data into numeric vs. categorical
users_numeric = users[numeric_cols].copy()
users_categorical = users[categorical_cols].copy()
users_categorical['City'] = users_categorical['City'].apply(lambda s: np.nan if s == 'nan' else s)
users_categorical['Country'] = users_categorical['City'].apply(lambda s: np.nan if s == 'nan' else s)

# 2) Impute numeric columns with IterativeImputer
imputer_num = IterativeImputer(random_state=0)
imputed_numeric = imputer_num.fit_transform(users_numeric)
imputed_numeric_df = pd.DataFrame(imputed_numeric, columns=numeric_cols)

# 3) Impute categorical columns with SimpleImputer (using the most frequent value)
imputer_cat = SimpleImputer(strategy='most_frequent')
imputed_categorical = imputer_cat.fit_transform(users_categorical)
imputed_categorical_df = pd.DataFrame(imputed_categorical, columns=categorical_cols)

# Ensure categorical columns are strings (they should be after imputation)
imputed_categorical_df = imputed_categorical_df.astype(str)

# 4) Combine numeric and categorical DataFrames
final_df = pd.concat([imputed_numeric_df, imputed_categorical_df], axis=1)

# Optionally convert numeric columns to integer type
final_df['User-ID'] = final_df['User-ID'].astype(int)
final_df['Age'] = final_df['Age'].astype(int)

users = final_df.copy()
users.head()

,User-ID,Age,City,Country
0,1,34,New York,New York
1,2,18,Stockton,Stockton
2,3,34,Moscow,Moscow
3,4,17,Porto,Porto
4,5,34,Farnborough,Farnborough


In [271]:
pd.DataFrame(users.isna().sum() / users.shape[0], columns=['Pct Missing'])

,Pct Missing
User-ID,0.0
Age,0.0
City,0.0
Country,0.0


In [272]:
# Fix weird string issue
users.loc[users['Country'].str.contains('SRI LANKA', na=False), 'Country'] = 'SRI LANKA'

# Export cleaned data

In [274]:
books.to_csv('books_cleaned.csv', header=True, index=None)
book_ratings.sample(n=book_ratings.shape[0]//5, random_state=42).to_csv('book_ratings_cleaned.csv', header=True, index=None)
users.to_csv('users_cleaned.csv', header=True, index=None)